In [57]:
## TSV-Analyzer-Iteration.ipynb

import os
import time
import pandas as pd
import numpy as np

In [58]:
#C:\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: 
### DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
### has_raised = await self.run_ast_nodes(code_ast.body, cell_name,

In [59]:
# measure wall time
t0 = time.time()

In [60]:
os.getcwd()
%store -r filepath
%store -r output_excel

#filepath='C:\\CPP_Performance\\TSV Files\\UAEdits\\6D105A9CB9AB46169A5AB5B7A4FA1C1B7A35CF4E48E90C41 - 2021-06-16 09-57-11-470.tsv'
#output_excel='C:\\CPP_Performance\\TSV Files\\UAEdits\\6D105A9CB9AB46169A5AB5B7A4FA1C1B7A35CF4E48E90C41 - 2021-06-16 09-57-11-470.xlsx'

#filepath='C:\\CPP_Performance\\TSV Files\\Parkland School Policy\\NovReleaseTSV\\2CE2AF537D274279B8680BF7C3ED40977A35CF4E48E90C41 - 2021-11-03 14-01-05-228.tsv'
#output_excel='C:\\CPP_Performance\\TSV Files\\Parkland School Policy\\NovReleaseTSV\\2CE2AF537D274279B8680BF7C3ED40977A35CF4E48E90C41 - 2021-11-03 14-01-05-228.xlsx'

#filepath='C:\\CPP_Performance\\TSV Files\\CLMQA\\CD4CCA2343B44EEAB4932B20DEE7EB3A79272D7148E90C41 - 2021-10-27 16-09-57-931.tsv'
#output_excel='C:\\CPP_Performance\\TSV Files\\CLMQA\\CD4CCA2343B44EEAB4932B20DEE7EB3A79272D7148E90C41 - 2021-10-27 16-09-57-931.xlsx'

#filepath="C:\\CPP_Performance\\TSV Files\\OffShoreTSV\\Divyaraj\\Occupancy\\After_TSVDebug - 2021-11-03 08-56-25-777.TSV"
#output_excel="C:\\CPP_Performance\\TSV Files\\OffShoreTSV\\Divyaraj\\Occupancy\\After_TSVDebug - 2021-11-03 08-56-25-777.xlsx"

#filepath='C:\\CPP_Performance\\TSV Files\\Staging\\7DA7B8CBD58E425EBFDB0BA087C682F179272D7148E90C41 - 2021-10-27 14-55-26-997.tsv'
#output_excel='C:\\CPP_Performance\\TSV Files\\Staging\\7DA7B8CBD58E425EBFDB0BA087C682F179272D7148E90C41 - 2021-10-27 14-55-26-997.xlsx'

print('starting to read file ', filepath)

df = pd.read_csv(filepath, sep='~', engine='c', header=1
                ,usecols=['begin_end', 'debugID', 'startTime', 'tagName', 'idref', 'level']
                ,names=['begin_end', 'debugID', 'startTime', 'tagName', 'idref', 'level']
                ,dtype= {'begin_end': 'category', 'debugID': str, 'startTime': str, 'tagName':'category', 'idref':str, 'level':'category'}
                )
                         #, np.float64, ‘b’: np.int32, ‘c’: ‘Int64’}

starting to read file  C:\CPP_Performance\TSV Files\CLMQA\CD4CCA2343B44EEAB4932B20DEE7EB3A79272D7148E90C41 - 2021-10-27 16-09-57-931.tsv


In [5]:
t1 = (time.time() - t0, "seconds wall time after read_csv")
print(t1)
#df.describe

(6.953732490539551, 'seconds wall time after read_csv')


In [6]:
#df.columns = ['begin_end', 'debugID', 'startTime', 'tagName', 'idref']
#df.head(10)

In [7]:
# sanitize data
df.drop(df[df['debugID'] == ' unique debugID '].index, inplace = True)
# convert str to int
df['debugID'] = pd.to_numeric(df['debugID'])

df.head()

,begin_end,debugID,startTime,tagName,idref,level
0,b,1.0,09:57:11:470,page,NaN,0
1,b,2.0,09:57:11:470,group,NaN,1
2,b,3.0,09:57:11:470,minOccurs,NaN,2
3,e,3.0,09:57:11:470,1,NaN,NaN
4,b,4.0,09:57:11:470,maxOccurs,NaN,2


In [8]:
t2 = (time.time() - t0, "seconds wall time after sanitization")
print(t2)


(13.222415924072266, 'seconds wall time after sanitization')


In [9]:
df1=df.groupby(['idref']).count().sort_values(by='debugID', ascending=False)['tagName']
df1.head(20)


idref
SelectedOccupancyInput.OccupancyID                     92694
SelectedOccupancyPrivate.SetSelectedCSPXPath           58624
RiskInput.Type                                         50279
True                                                   45664
Occupancy                                              43626
BuildingDetailPage.SetLastSelectedOccupancyID          42871
BuildingInput.OccupancyID                              29753
SelectedOccupancyPrivate.SetSelectedFullDescription    26622
RiskPrivate.SpecialRiskCheck                           20772
RiskInput.OccupancyID                                  19833
OccupancySearchInputNonShredded.SearchResults          19589
RiskInput.LocationID                                   18553
False                                                  18286
RiskInput.BuildingID                                   18176
SelectedOccupancyPrivate.SetKeyValue                   17748
OccupancyOutput.CSP                                    16379
OccupancyInput.CSP

In [10]:
df1.describe

<bound method NDFrame.describe of idref
SelectedOccupancyInput.OccupancyID                                 92694
SelectedOccupancyPrivate.SetSelectedCSPXPath                       58624
RiskInput.Type                                                     50279
True                                                               45664
Occupancy                                                          43626
                                                                   ...  
UNIG_PolicyUndRulesPrivate.ShowUnderwritingGuidelines                  1
CovEndorsements                                                        1
UNIG_PolicyUndRulesPrivate.ShowWindOrHailGuidelinesResult              1
UNIG_PolicyUndRulesPrivate.ShowWindOrHailGuidelinesResultPhase3        1
LocationInlandMarineIRPMOutput.LOB                                     1
Name: tagName, Length: 1587, dtype: int64>

In [11]:
# export to Excel ####
with pd.ExcelWriter(output_excel) as writer:  
    df1.to_excel(writer, sheet_name='frequency', header=['count'])


In [12]:
del df1

In [13]:
t3 = (time.time() - t0, "seconds wall time after field frequency")
print(t3)

(14.842190980911255, 'seconds wall time after field frequency')


In [14]:
#df[df.idref.isin(['LocationPrivate.CheckBuildingLimit', 'BuildingDetailPage.SetLastSelectedOccupancyID', 'OccupancyOutputNonShredded.ApplicableToProperty'])].groupby('idref').count()



In [15]:
df1 = df[df.tagName.isin(['iterator'])].groupby(['idref'])['idref','debugID'].count().sort_values(by='debugID', ascending=False)['idref']
df1.columns=['iterator', 'count']


<ipython-input-15-56c29cfd0210>:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  df1 = df[df.tagName.isin(['iterator'])].groupby(['idref'])['idref','debugID'].count().sort_values(by='debugID', ascending=False)['idref']


In [16]:
# export to Excel ####
with pd.ExcelWriter(output_excel, engine="openpyxl", mode='a') as writer:  
    df1.to_excel(writer, sheet_name='iterator-Group', header=['count'])


In [17]:
del df1
t4 = (time.time() - t0, "seconds wall time after iterator")
print(t4)

(16.209899187088013, 'seconds wall time after iterator')


In [18]:
# count levels
#df.groupby(['level']).count().sort_values(by='level', ascending=False)

In [19]:
# elapsed time of public/private fields


In [20]:
# list debugID for public/private idref - both begin and end lines
debugList = df[df.tagName.isin(['public', 'private'])].debugID.tolist()
fDebugVal = list(map(int, debugList))


In [21]:
print('public/private count:', len(fDebugVal))
type(fDebugVal)
print(fDebugVal[1:20])
#df[df.debugID=='11']
dfb = df[df.debugID.isin(fDebugVal)].sort_values(by='debugID')


public/private count: 540501
[16, 24, 31, 34, 38, 46, 50, 66, 71, 76, 80, 86, 89, 92, 96, 101, 107, 110, 113]


In [22]:
dfbstart = dfb[dfb.begin_end=='b'][['debugID', 'startTime', 'idref', 'level']]
dfbend = dfb[dfb.begin_end=='e'][['debugID', 'startTime']]
dfbend.columns=['debugID', 'endTime']


In [23]:
dfbstart

,debugID,startTime,idref,level
19,13.0,09:57:11:470,PolicyPrivate.UNIG_UnderwritingStatusCaption,5
22,16.0,09:57:11:470,PolicyOutput.UnderwritingStatus,8
34,24.0,09:57:11:470,PolicyOutputNonShredded.LastTransactionStatus,12
45,31.0,09:57:11:470,TransactionOutput.Status,15
61,34.0,09:57:11:470,LineOutputNonShredded.UNIG_ShowIssuePolicyPage,5
...,...,...,...,...
5437496,2718751.0,09:59:12:757,Context.IsAgent,14
5437503,2718754.0,09:59:12:757,PolicyOutputNonShredded.UNIG_SubmissionComplete,13
5437512,2718758.0,09:59:12:757,Compro.UNIG_ComproError,12
5437520,2718761.0,09:59:12:757,UNIG_PolicyUndQuestionsInput.showUpdateUnderMe...,10


In [24]:
dfbend

,debugID,endTime
58,13.0,09:57:11:470
23,16.0,09:57:11:470
49,24.0,09:57:11:470
46,31.0,09:57:11:470
148,34.0,09:57:11:470
...,...,...
5437497,2718751.0,09:59:12:757
5437504,2718754.0,09:59:12:757
5437513,2718758.0,09:59:12:757
5437529,2718761.0,09:59:12:757


In [25]:
dfcombined=pd.merge(dfbend, dfbstart, on=["debugID"])
dfcombined[dfcombined.idref=='LocationPrivate.CheckBuildingLimit']
#dfb.sort_values(by='debugID')
#dfb[dfb.debugID==79806].startTime.tolist()

#dfb[dfb.debugID==11]

,debugID,endTime,startTime,idref,level
1638,12548.0,09:57:44:811,09:57:44:718,LocationPrivate.CheckBuildingLimit,57
2449,18915.0,09:57:45:233,09:57:44:811,LocationPrivate.CheckBuildingLimit,57
6345,46869.0,09:57:45:312,09:57:45:233,LocationPrivate.CheckBuildingLimit,57
7011,52688.0,09:57:45:344,09:57:45:312,LocationPrivate.CheckBuildingLimit,57
7304,55115.0,09:57:45:470,09:57:45:344,LocationPrivate.CheckBuildingLimit,57


In [26]:
dfb[:10]

,begin_end,debugID,startTime,tagName,idref,level
19,b,13.0,09:57:11:470,private,PolicyPrivate.UNIG_UnderwritingStatusCaption,5
58,e,13.0,09:57:11:470,Inforce,NaN,NaN
22,b,16.0,09:57:11:470,public,PolicyOutput.UnderwritingStatus,8
23,e,16.0,09:57:11:470,InForce,NaN,NaN
34,b,24.0,09:57:11:470,public,PolicyOutputNonShredded.LastTransactionStatus,12
49,e,24.0,09:57:11:470,Referred,NaN,NaN
45,b,31.0,09:57:11:470,public,TransactionOutput.Status,15
46,e,31.0,09:57:11:470,Referred,NaN,NaN
61,b,34.0,09:57:11:470,private,LineOutputNonShredded.UNIG_ShowIssuePolicyPage,5
148,e,34.0,09:57:11:470,(false),NaN,NaN


In [27]:
dfcombined['timediff'] = pd.DataFrame(( pd.to_datetime(dfcombined['endTime'], format='%H:%M:%S:%f') - pd.to_datetime(dfcombined['startTime'], format='%H:%M:%S:%f')).dt.total_seconds())

In [28]:
from datetime import datetime, timedelta
df1 = dfcombined[dfcombined.timediff > 0.050].sort_values(by='timediff', ascending=False)


In [29]:
# export to Excel ####
with pd.ExcelWriter(output_excel, engine="openpyxl", mode='a') as writer:  
    df1.to_excel(writer, sheet_name='timing-DebugID', header=['debugID','endTime','startTime','idref','level','timediff (sec)'])


In [30]:
df1

,debugID,endTime,startTime,idref,level,timediff
1163,9152.0,09:59:11:380,09:57:42:108,UNIG_PolicyUndRulesPrivate.NoCautionsFound,13,89.272
1162,9148.0,09:59:11:380,09:57:42:108,PolicyOutputNonShredded.UNIG_UnderwritingRuleO...,9,89.272
7824,59983.0,09:59:11:334,09:57:45:470,UNIG_PolicyUndRulesPrivate.NoCautionsBuildingL...,19,85.864
7825,59987.0,09:59:11:334,09:57:45:470,UNIG_PolicyUndRulesPrivate.ThereAreBuildingUAE...,23,85.864
7826,59991.0,09:59:11:334,09:57:45:470,LinePrivate.UNIG_AnyBuildingUAEdits,27,85.864
...,...,...,...,...,...,...
165473,841546.0,09:58:14:295,09:58:14:242,BuildingPrivate.CalculateCSPUAEdit,50,0.053
270484,1366958.0,09:58:32:345,09:58:32:294,BuildingPrivate.CalculateCSPUAEdit,50,0.051
142152,727990.0,09:58:10:090,09:58:10:039,BuildingPrivate.CalculateCSPUAEdit,50,0.051
1642,12564.0,09:57:44:769,09:57:44:718,BuildingPrivate.CheckBuildingRiskLimit,60,0.051


In [31]:
df2 = dfcombined.groupby('idref').sum('timediff').sort_values(by='timediff', ascending=False)
df2 = df2[df2.timediff > 0.050][['timediff']]
df2

,timediff
idref,
UNIG_PolicyUndRulesPrivate.NoCautionsFound,89.272
PolicyOutputNonShredded.UNIG_UnderwritingRuleOutcome,89.272
UNIG_PolicyUndRulesPrivate.ThereAreBuildingUAEdits,85.864
LinePrivate.UNIG_AnyBuildingUAEdits,85.864
UNIG_PolicyUndRulesPrivate.NoCautionsBuildingLevelFound,85.864
...,...
LocationPropertyIRPMOutput.BackupTotal,0.057
LocationPropertyIRPMPrivate.IsPropertyIRPMCommentsFactorEnteredCheck,0.054
LocationPropertyIRPMOutput.CommentsStatus,0.054


In [32]:
# export to Excel ####
with pd.ExcelWriter(output_excel, engine="openpyxl", mode='a') as writer:  
    df2.to_excel(writer, sheet_name='timing-Unique-fields')


In [33]:
del dfcombined
del df1
del df2

In [34]:
t5 = (time.time() - t0, "seconds wall time after timing")
print(t5)

(20.49694538116455, 'seconds wall time after timing')


In [35]:
# list line index for iterator idref - both begin and end lines
iterList = df[df.tagName.isin(['iterator'])].index.tolist()

iterDebugList = df.iloc[iterList]['debugID'].tolist()
iterList[:10], iterDebugList[:10]

([36, 178, 212, 294, 530, 638, 704, 726, 748, 770],
 [26.0, 98.0, 115.0, 160.0, 273.0, 332.0, 365.0, 376.0, 387.0, 398.0])

In [36]:
# Find the immediate parent of the iterator
# Iterator debugID - walk backward till we find public/private




In [37]:
# list line index for public/private idref - both begin and end lines
indexList = df[df.tagName.isin(['public', 'private'])].index.tolist()

df.iloc[indexList[:5]]

,begin_end,debugID,startTime,tagName,idref,level
19,b,13.0,09:57:11:470,private,PolicyPrivate.UNIG_UnderwritingStatusCaption,5
22,b,16.0,09:57:11:470,public,PolicyOutput.UnderwritingStatus,8
34,b,24.0,09:57:11:470,public,PolicyOutputNonShredded.LastTransactionStatus,12
45,b,31.0,09:57:11:470,public,TransactionOutput.Status,15
61,b,34.0,09:57:11:470,private,LineOutputNonShredded.UNIG_ShowIssuePolicyPage,5


In [38]:
# count of the iterator index within specific iterator

b1 = max([i for i in indexList if i < 178])
b1

b2 = min([i for i in reversed(indexList) if i > 178])
b2

print(b1, b2)

176 183


In [39]:
idx1 = indexList.copy()
len(idx1)/10000, len(iterList)

(54.0501, 55765)

parent = lambda x: [i for i in idx1 if i < x]
child = lambda x: [i for i in reversed(idx1) if i > x]
#parent(36), iterList[-1], child(iterList[-1])

In [40]:
parent2 = lambda x, y: [i for i in y if i < x]
child2 = lambda x, y: [i for i in y if i > x]

iterList[:3],parent2(36, idx1), min(child2(iterList[-1], reversed(idx1)))

([36, 178, 212], [19, 22, 34], 5437412)

In [41]:
# if the element is not in the source list
#remv = lambda x, y: [i for i in y if not i in x or x.remove(i)]

# we know all elements are available in the source list
remv = lambda x, y: [i for i in y if x.remove(i)]

# we can slice the list using len
remv2 = lambda x, y: x[len(y):]

idx1 = indexList.copy()
len(parent2(iterList[3], idx1)), idx1[:10], remv(idx1, parent2(36, idx1))[:10]
#iterList[:3], idx1[:10]

(29, [19, 22, 34, 45, 61, 65, 79, 83, 109, 121], [])

In [42]:
#idx1 = indexList.copy()
#bb = lambda x: [max([parent(i, idx1) and remv(idx1, parent(i, idx1)) for i in x ])]
#bb = lambda x: [max(parent2(i, idx1) + remv(idx1, parent2(i, idx1)))  for i in x ]
#bb(iterList[:500])[:5]

In [43]:
max([0]), len(iterList)

(0, 55765)

In [44]:
#brute force search
def fieldList(iterList, idxList):
    fieldList = []
    for item in iterList:
        p = parent2(item, idxList)
        if len(p) > 0:
            fieldList.append(max(p))
            idxList = idxList[len(p):]
        #print(p, fieldList)

    return fieldList                   

In [45]:
#mini-batch search
def fieldList(itrList, idxList, parent=True):
    fieldList = []
    fieldArray = []
    
    batchsize = 10000
    if not parent:
        itrList.reverse()
        idxList.reverse()

    # measure wall time
    t0 = time.time()

    #print(len(idxList))
    # mini-batch to traverse indexList
    while len(idxList) > 0:
        # slicing idxList by batchsize
        idx1 = idxList[:min(len(idxList), batchsize)]
        cntr = 0
        for item in itrList:
            if parent:
                pc = parent2(item, idx1)
            else:
                pc = child2(item, idx1)
            
            if len(pc) > 0:
                # capture array of parent/children from last iterator
                fieldArray.append(pc)
                if parent:
                    fieldList.append(max(pc))
                else:
                    fieldList.append(min(pc))
                idx1 = idx1[len(pc):]
                cntr+=1
            else:
                #print('no parent for item ', item)
                break

        #print (time.time() - t0, "seconds elapsed...", len(idxList) / batchsize, len(idxList), len(itrList))

        # reset idxList by slicing the head, retaining the tail by batchsize
        idxList = idxList[min(len(idxList), batchsize):]
        # reset  iterList by slicing the head, retaining the tail by cntr
        itrList = itrList[cntr:]
        if len(itrList) < 1:
            break
        #if parent:
         #   print(item, max(fieldList))
        #else:
         #   print(item, min(fieldList))
            
    return fieldList #, fieldArray

In [46]:
idx1 = indexList.copy()
iter1 = iterList.copy()
f = fieldList(iter1, idx1, parent=True)


len(indexList), len(iterList), len(f), len(i)

i[:5], f[:5], iter1[:20]

df.iloc[i]['idref']

In [47]:
df1=df.iloc[f][['begin_end', 'debugID', 'startTime', 'tagName', 'idref', 'level']]
df1.reset_index(drop=True, inplace=True)
df2=df.loc[iterList]['idref']
df2.columns=['iterator']
df2.reset_index(drop=True, inplace=True)
df1, df2
dfcombined=pd.concat([df1, df2], axis=1, join="inner")
del df1, df2
#len(df2), df1, df.loc[i]
#df1['iterator']=pd.DataFrame(df2)
#df1

In [48]:
dfcombined.columns=['begin_end', 'debugID', 'startTime', 'tagName', 'idref', 'level', 'iterator']

dfcombined


,begin_end,debugID,startTime,tagName,idref,level,iterator
0,b,24.0,09:57:11:470,public,PolicyOutputNonShredded.LastTransactionStatus,12,Transaction
1,b,96.0,09:57:11:470,public,PolicyOutputNonShredded.UNIG_LastTransactionTy...,14,Transaction
2,b,113.0,09:57:11:470,private,PolicyPrivate.UNIG_LastTransactionStatusForDis...,14,Transaction
3,b,158.0,09:57:11:470,private,PolicyPrivate.UNIG_LastTransactionStatusForDis...,22,Transaction
4,b,271.0,09:57:11:470,private,ModifyProcessingData.TransactionCount,12,Transaction_Current
...,...,...,...,...,...,...,...
55760,b,2718288.0,09:59:12:741,public,PolicyOutputNonShredded.LastTransactionStatus,28,Transaction
55761,b,2718299.0,09:59:12:741,public,PolicyOutputNonShredded.LastTransactionStatus,28,Transaction
55762,b,2718310.0,09:59:12:741,public,PolicyOutputNonShredded.LastTransactionStatus,28,Transaction
55763,b,2718327.0,09:59:12:741,private,PolicyOutputNonShredded.LastTransactionHistoryID,28,Transaction


In [49]:
# export to Excel ####
with pd.ExcelWriter(output_excel, engine="openpyxl", mode='a') as writer:  
    dfcombined.to_excel(writer, sheet_name='field-iteration', header=['begin_end', 'debugID','startTime','tagName','idref','level','iterator group'])


In [50]:
t6 = (time.time() - t0, "seconds wall time after iterator field")
print(t6)

(63.501057386398315, 'seconds wall time after iterator field')


idx1 = indexList.copy()
iter1 = iterList.copy()

b = fieldList(iter1, idx1, parent=False)
b.reverse()
#iterList.reverse()



#df.iloc[indexList[:10]][['debugID', 'tagName']], df.iloc[iterList[:10]][['debugID', 'tagName']]
#df.iloc[b[55:59]]

#df.iloc[f[:5]][['debugID', 'tagName']], df.iloc[b[:5]][['debugID', 'tagName']]
#df.iloc[b[55:59]]

print(iterList[:2], iterList[-1], indexList[:2], indexList[-1])


In [51]:
df1=dfcombined.groupby(['idref', 'iterator']).count().sort_values(by='tagName', ascending=False)['tagName']
df1[:10]


idref                                                        iterator 
SelectedOccupancyPrivate.SetSelectedFullDescription          Occupancy    13299
SelectedOccupancyPrivate.SetSelectedCSP                      Occupancy     4516
SelectedOccupancyPrivate.SetSelectedClassLimit               Occupancy     4514
SelectedOccupancyPrivate.SetSelectedOccupancyType            Occupancy     4511
SelectedOccupancyPrivate.SetSelectedPropertyDescription      Occupancy     4436
SelectedOccupancyPrivate.SetSelectedProtectionSafeguard      Occupancy     4435
SelectedOccupancyPrivate.SetSelectedHazardGrade              Occupancy     4435
SelectedOccupancyPrivate.SetSelectedFullDescriptionRiskType  Occupancy     2608
BuildingPrivate.IterateRiskAndCheckBuildingRiskCount         Risk           841
BuildingOutputNonShredded.UAEditTIVLimit                     Risk           841
Name: tagName, dtype: int64

In [52]:
# export to Excel ####
with pd.ExcelWriter(output_excel, engine="openpyxl", mode='a') as writer:  
    df1.to_excel(writer, sheet_name='iterator-Field', header=['count'])


In [53]:
t7 = (time.time() - t0, "seconds wall time after iterator field count")
print(t7)

(88.1357524394989, 'seconds wall time after iterator field count')


In [54]:
print(t1)
print(t2)
print(t3)
print(t4)
print(t5)
print(t6)
print(t7)


(6.953732490539551, 'seconds wall time after read_csv')
(13.222415924072266, 'seconds wall time after sanitization')
(14.842190980911255, 'seconds wall time after field frequency')
(16.209899187088013, 'seconds wall time after iterator')
(20.49694538116455, 'seconds wall time after timing')
(63.501057386398315, 'seconds wall time after iterator field')
(88.1357524394989, 'seconds wall time after iterator field count')


In [55]:
##### capture iterator debugID for all nested public/private fields 
##### helps identify impact of fields by their respective debugID based on iteration counts



In [56]:
##### capture level for a nested operation to establish pattern
####- helps identify repeating patterns across policy
